# Hotel-ID 2021 FGVC8 — Plan

Goal: Build a medal-winning baseline fast; iterate with solid CV and GPU-accelerated training.

Plan:
- Environment check: verify GPU (nvidia-smi) and install correct CUDA 12.1 PyTorch if needed.
- Data audit:
  - Inspect train.csv schema, class counts, and image availability under train_images/*.
  - Verify test_images/ count and sample_submission format.
- Validation:
  - Stratified KFold by hotel_id; ensure no leakage. Use deterministic seeds.
- Baseline model:
  - torchvision ImageNet-pretrained backbone (e.g., tf_efficientnet_b0 or resnet50; if no timm preinstalled, use torchvision resnet50).
  - Image size 224-256; augmentations (flip, color jitter, random resized crop).
  - Optimizer AdamW, cosine schedule, label smoothing; focal loss optional.
  - FP16 mixed precision with GradScaler.
- Inference:
  - TTA (horizontal flip) and average logits.
  - Produce top-5 per image in required submission format.
- Iterations:
  - If baseline is weak, switch to timm and larger backbones, mixup/cutmix; tune image size and epochs.
  - Cache datasets and OOF logits; try multiple seeds and blend.

Requests to experts will be made after: plan, data audit, baseline CV, first LB score, and major changes.

In [1]:
import os, sys, time, json, shutil, subprocess, random, textwrap
import pandas as pd
from pathlib import Path

print('=== GPU CHECK ===', flush=True)
try:
    out = subprocess.run(['bash','-lc','nvidia-smi || true'], capture_output=True, text=True, check=False)
    print(out.stdout)
except Exception as e:
    print('nvidia-smi failed:', e)

print('=== CWD and Files ===', flush=True)
cwd = Path('.').resolve()
print('CWD:', cwd)
for p in ['train_images','test_images','train.csv','sample_submission.csv','description.md']:
    print(p, 'exists:', Path(p).exists())

print('\n=== Data Audit ===', flush=True)
train_csv = pd.read_csv('train.csv')
print('train.csv shape:', train_csv.shape)
print('train.csv columns:', list(train_csv.columns))
print(train_csv.head(3))
if 'hotel_id' in train_csv.columns:
    print('n_unique hotel_id:', train_csv['hotel_id'].nunique())
    print('top hotel_id counts:\n', train_csv['hotel_id'].value_counts().head(5))

sample_sub = pd.read_csv('sample_submission.csv')
print('\nsample_submission shape:', sample_sub.shape)
print(sample_sub.head(3))

print('\n=== Images Count ===', flush=True)
test_dir = Path('test_images')
train_dir = Path('train_images')
def count_images(path: Path):
    exts = {'.jpg','.jpeg','.png','.bmp','.webp'}
    c = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            if Path(f).suffix.lower() in exts:
                c += 1
    return c
print('test_images count:', count_images(test_dir))
print('train_images subdirs:', len([d for d in train_dir.iterdir() if d.is_dir()]))
print('train_images total images (may take a moment)...')
t0=time.time()
train_img_count = count_images(train_dir)
print('train_images count:', train_img_count, 'elapsed:', round(time.time()-t0,2),'s')

print('\n=== Verify file existence for sample rows ===', flush=True)
exts = {'.jpg','.jpeg','.png','.bmp','.webp'}
def find_train_path(row):
    # common patterns:
    # 1) train_images/<hotel_id>/<image>
    # 2) train_images/<image>
    img = str(row.get('image', row.get('image_id', '')))
    hid = str(row.get('hotel_id', ''))
    candidates = []
    if img:
        candidates.append(train_dir/img)
        if hid:
            candidates.append(train_dir/hid/img)
    for c in candidates:
        if c.exists():
            return str(c)
    # fallback: search only within the hotel_id folder if exists, try any extension
    if hid and (train_dir/hid).exists():
        for e in exts:
            p = train_dir/hid/(img if Path(img).suffix else img+e)
            if p.exists():
                return str(p)
    return None

samples = train_csv.sample(min(5, len(train_csv)), random_state=42)
found = 0
for _, r in samples.iterrows():
    p = find_train_path(r)
    print('image lookup:', {'row_image': r.get('image', r.get('image_id', '')), 'hotel_id': r.get('hotel_id', None), 'path': p})
    if p: found += 1
print(f'Found {found}/{len(samples)} sample image files')

print('\n=== Test sample ===', flush=True)
test_files = sorted([p.name for p in test_dir.iterdir() if p.is_file()])[:5]
print('test_images sample files:', test_files)

print('\nAudit complete.')

=== GPU CHECK ===


Sat Sep 27 05:06:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

CWD: /var/lib/simon/agent_run_states/hotel-id-2021-fgvc8-20250927-045136
train_images exists: True
test_images exists: True
train.csv exists: True
sample_submission.csv exists: True
description.md exists: True

=== Data Audit ===


train.csv shape: (87798, 4)
train.csv columns: ['image', 'chain', 'hotel_id', 'timestamp']
                  image  chain  hotel_id            timestamp
0  d29287f52c2a871f.jpg      5     22408  2018-04-16 17:01:49
1  e9d067c249e4c2f9.jpg     70      2324  2016-07-08 22:26:21
2  cc9877a40a63ed93.jpg      4     47514  2017-04-14 02:28:56
n_unique hotel_id: 7770
top hotel_id counts:
 hotel_id
36363    86
18807    84
60181    81
53586    80
64314    77
Name: count, dtype: int64

sample_submission shape: (9756, 2)
                  image                       hotel_id
0  f1608c9f17fb6920.jpg  36363 53586 18807 64314 60181
1  c6c63939c67931e1.jpg  36363 53586 18807 64314 60181
2  83c214f3e90717ed.jpg  36363 53586 18807 64314 60181

=== Images Count ===


test_images count: 9756
train_images subdirs: 88
train_images total images (may take a moment)...


train_images count: 87797 elapsed: 0.18 s

=== Verify file existence for sample rows ===


image lookup: {'row_image': 'b3b34c5d6186b362.jpg', 'hotel_id': 61722, 'path': None}
image lookup: {'row_image': 'cdf882b7489fa4d0.jpg', 'hotel_id': 27853, 'path': None}
image lookup: {'row_image': '99c98c732e994ea5.jpg', 'hotel_id': 33691, 'path': None}
image lookup: {'row_image': 'c93609d63d82b1cf.jpg', 'hotel_id': 28303, 'path': None}
image lookup: {'row_image': 'b595482aaf544db6.jpg', 'hotel_id': 45003, 'path': None}
Found 0/5 sample image files

=== Test sample ===


test_images sample files: ['80196e6999ce63cf.jpg', '80296afd55d516ea.jpg', '802aab95d62b7daa.jpg', '802af4d04faf14df.jpg', '802b5ed622fd3587.jpg']

Audit complete.


In [2]:
from pathlib import Path
import pandas as pd

print('=== Build file paths using chain directory ===', flush=True)
train_df = pd.read_csv('train.csv')
train_df['chain'] = train_df['chain'].astype(str)
train_df['file_path'] = 'train_images/' + train_df['chain'] + '/' + train_df['image'].astype(str)
train_df['exists'] = train_df['file_path'].apply(lambda p: Path(p).exists())
coverage = train_df['exists'].mean()
missing = (~train_df['exists']).sum()
print(f'train path coverage: {coverage:.4f} ({len(train_df)-missing}/{len(train_df)}) missing: {missing}')
if missing > 0:
    print('Sample missing rows:')
    print(train_df.loc[~train_df['exists'], ['image','chain','hotel_id']].head(5))

train_df = train_df.loc[train_df['exists']].reset_index(drop=True)
print('Kept rows after drop missing:', len(train_df))

print('\n=== Build test file paths ===', flush=True)
test_dir = Path('test_images')
test_files = sorted([p.name for p in test_dir.iterdir() if p.is_file()])
test_df = pd.DataFrame({'image': test_files})
test_df['file_path'] = test_df['image'].apply(lambda x: str(test_dir / x))
print('Test count:', len(test_df))
print(test_df.head(3))

print('\nReady for embedding extraction next.')

=== Build file paths using chain directory ===


train path coverage: 1.0000 (87798/87798) missing: 0
Kept rows after drop missing: 87798

=== Build test file paths ===


Test count: 9756
                  image                         file_path
0  80196e6999ce63cf.jpg  test_images/80196e6999ce63cf.jpg
1  80296afd55d516ea.jpg  test_images/80296afd55d516ea.jpg
2  802aab95d62b7daa.jpg  test_images/802aab95d62b7daa.jpg

Ready for embedding extraction next.


In [3]:
import os, sys, subprocess, shutil
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

print('=== Install CUDA 12.1 PyTorch stack and deps ===', flush=True)
# Uninstall any preexisting torch stack to avoid conflicts
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean potential shadow dirs (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
):
    if os.path.exists(d):
        print('Removing', d, flush=True)
        shutil.rmtree(d, ignore_errors=True)

# Install exact cu121 wheels
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Write constraints to pin torch versions for subsequent installs
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

print('=== Install non-torch deps (timm, faiss-cpu, pillow, opencv) ===', flush=True)
pip('install', '-c', 'constraints.txt',
    'timm==1.0.9',
    'faiss-cpu==1.8.0.post1',
    'opencv-python-headless==4.10.0.84',
    'pillow==10.4.0',
    '--upgrade-strategy', 'only-if-needed')

import torch, torchvision
print('torch:', torch.__version__, 'CUDA build:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

import timm, faiss
print('timm:', timm.__version__)
print('faiss:', getattr(faiss, '__version__', 'unknown'))
print('Install check complete.')

=== Install CUDA 12.1 PyTorch stack and deps ===


> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 240.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 181.8 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 203.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 476.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 490.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 428.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 425.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 473.7 MB/s eta 0:00:00


torch: 2.4.1+cu121 CUDA build: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
timm: 1.0.9
faiss: 1.8.0
Install check complete.


/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os, math, time, json, gc, sys
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps
import timm
from timm.data import create_transform
import faiss

print('=== Embedding extraction + FAISS retrieval (convnext_tiny, 384) ===', flush=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 384
BATCH_SIZE = 96  # safe for A10 24GB at 384
NUM_WORKERS = min(12, os.cpu_count() or 4)
USE_TTA = True  # hflip TTA
TOPK = 100
P_EXP = 1.5  # sim^p aggregation

# Load prepared dataframes (from previous cells)
train_df = pd.read_csv('train.csv')
train_df['chain'] = train_df['chain'].astype(str)
train_df['file_path'] = 'train_images/' + train_df['chain'] + '/' + train_df['image'].astype(str)
test_dir = Path('test_images')
test_files = sorted([p.name for p in test_dir.iterdir() if p.is_file()])
test_df = pd.DataFrame({'image': test_files})
test_df['file_path'] = test_df['image'].apply(lambda x: str(test_dir / x))

# Sanity
assert len(train_df) > 0 and len(test_df) > 0

class ImageDataset(Dataset):
    def __init__(self, file_paths, transform):
        self.file_paths = list(file_paths)
        self.transform = transform
    def __len__(self):
        return len(self.file_paths)
    def __getitem__(self, idx):
        fp = self.file_paths[idx]
        img = Image.open(fp).convert('RGB')
        img = ImageOps.exif_transpose(img)
        x = self.transform(img)
        return x

def build_transform(img_size=384):
    return create_transform(input_size=img_size, is_training=False)

@torch.no_grad()
def extract_embeddings(model, file_paths, tta=False, batch_size=BATCH_SIZE):
    model.eval()
    tfm = build_transform(IMG_SIZE)
    ds = ImageDataset(file_paths, tfm)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    all_embs = []
    t0 = time.time()
    n = len(ds)
    for bi, batch in enumerate(dl):
        with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
            feats = model(batch.to(device))
        if tta:
            # horizontal flip TTA
            with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
                feats_flipped = model(torch.flip(batch, dims=[3]).to(device))
            feats = 0.5 * (feats + feats_flipped)
        feats = F.normalize(feats.float(), dim=1).cpu().numpy()
        all_embs.append(feats)
        if (bi+1) % 50 == 0 or (bi+1)==len(dl):
            elapsed = time.time()-t0
            done = (bi+1)*batch.size(0)
            print(f'  Emb {done}/{n} ({(done/n)*100:.1f}%) elapsed {elapsed:.1f}s', flush=True)
    embs = np.concatenate(all_embs, axis=0)
    return embs

# Create model (global_pool='avg' and num_classes=0 yields feature vectors)
print('Creating model...', flush=True)
model = timm.create_model('convnext_tiny', pretrained=True, num_classes=0, global_pool='avg')
model.to(device)
model.eval()
feat_dim = model(torch.zeros(1,3,IMG_SIZE,IMG_SIZE, device=device)).shape[1]
print('Feature dim:', feat_dim)

# Cache paths
train_emb_path = Path(f'train_emb_convnext_tiny_{IMG_SIZE}.npy')
test_emb_path = Path(f'test_emb_convnext_tiny_{IMG_SIZE}.npy')

# Train embeddings
if train_emb_path.exists():
    train_emb = np.load(train_emb_path)
    print('Loaded cached train embeddings:', train_emb.shape)
else:
    print('Extracting train embeddings...', flush=True)
    train_emb = extract_embeddings(model, train_df['file_path'].values, tta=USE_TTA)
    np.save(train_emb_path, train_emb)
    print('Saved train embeddings:', train_emb.shape)

# Test embeddings
if test_emb_path.exists():
    test_emb = np.load(test_emb_path)
    print('Loaded cached test embeddings:', test_emb.shape)
else:
    print('Extracting test embeddings...', flush=True)
    test_emb = extract_embeddings(model, test_df['file_path'].values, tta=USE_TTA)
    np.save(test_emb_path, test_emb)
    print('Saved test embeddings:', test_emb.shape)

del model; gc.collect();

# Ensure L2-normalized
def l2norm(x):
    x = x.astype('float32')
    faiss.normalize_L2(x)
    return x
train_emb = l2norm(train_emb)
test_emb = l2norm(test_emb)

print('Building FAISS index (IP over L2-normalized vectors -> cosine)...', flush=True)
index = faiss.IndexFlatIP(train_emb.shape[1])
index.add(train_emb)
print('Index ntotal:', index.ntotal)

print('Searching topK neighbors for all test...', flush=True)
t0=time.time()
sims, nbrs = index.search(test_emb, TOPK)
print('Search done in', round(time.time()-t0,2),'s')

# Aggregate per hotel_id
train_hids = train_df['hotel_id'].values.astype(np.int64)
preds_top5 = []
for i in range(len(test_df)):
    nb_idx = nbrs[i]
    nb_sim = sims[i]
    agg = {}
    for j in range(nb_idx.shape[0]):
        hid = int(train_hids[nb_idx[j]])
        w = float(nb_sim[j])
        if w <= 0: # neighbors can be negative with cosine; keep but small
            pass
        agg[hid] = agg.get(hid, 0.0) + (w ** P_EXP)
    # sort by score desc
    top = sorted(agg.items(), key=lambda x: x[1], reverse=True)[:5]
    top_ids = [str(h) for (h,_) in top]
    # pad if less than 5 (unlikely)
    while len(top_ids) < 5:
        top_ids.append(top_ids[-1] if top_ids else '0')
    preds_top5.append(' '.join(top_ids[:5]))
    if (i+1) % 500 == 0 or (i+1)==len(test_df):
        print(f'  Aggregated {i+1}/{len(test_df)}', flush=True)

# Build submission following sample order
sample = pd.read_csv('sample_submission.csv')
name_to_pred = dict(zip(test_df['image'].values, preds_top5))
sub = sample.copy()
sub['hotel_id'] = sub['image'].map(name_to_pred)
assert sub['hotel_id'].notnull().all(), 'Missing predictions for some images'
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape', sub.shape)
print(sub.head())
print('Pipeline complete.')

=== Embedding extraction + FAISS retrieval (convnext_tiny, 384) ===


Creating model...


Feature dim: 768
Extracting train embeddings...


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (108576768 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


/tmp/ipykernel_107/419949055.py:60: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):


...

  Aggregated 8500/9756


  Aggregated 9000/9756


  Aggregated 9500/9756


  Aggregated 9756/9756


Saved submission.csv with shape (9756, 2)
                  image                       hotel_id
0  f1608c9f17fb6920.jpg       782 15319 40842 492 5441
1  c6c63939c67931e1.jpg   30544 27517 37807 9492 26349
2  83c214f3e90717ed.jpg  14201 53242 54525 34150 16137
3  a6b5dbe436592c82.jpg   39435 6150 36307 13918 50333
4  dd2f02f4354993cc.jpg  15115 58441 60918 17911 62628
Pipeline complete.
